In [1]:
import pandas as pd

# Importing True news data

True_news=pd.read_csv(r"C:\Users\dhruv\Downloads\Datasets\NLP DATA\Fake News Detection\True.csv")
True_news.head(5)

True_news['Combined']= True_news['title'].astype(str)+" "+True_news['text'].astype(str)

True_news.duplicated().sum()
True_news.drop_duplicates(inplace=True)

True_news.isnull().sum()

# 1 is true 
True_news['label'] = 1

True_news['label'].value_counts()
# Class is balanced
True_news.head(10)

# Importing fake news data 

fake_news=pd.read_csv(r"C:\Users\dhruv\Downloads\Datasets\NLP DATA\Fake News Detection\Fake.csv")
fake_news.head(5)

fake_news['Combined']= fake_news['title'].astype(str)+" "+ fake_news['text'].astype(str)

fake_news.duplicated().sum()
fake_news.drop_duplicates(inplace=True)

fake_news.isnull().sum()

fake_news['label'] = 0
# 0 is fake 

fake_news['label'].value_counts()

# concatinating in one data frame 
df=pd.concat([True_news,fake_news],ignore_index=True)
df=df.sample(frac=1,random_state=42).reset_index(drop=True)
df.drop(df[['title','text','subject','date']],axis=1,inplace=True)

df['label'].value_counts()
# class is balanced



# preprocessing data 

df['Combined']=df['Combined'].str.lower()

# removing htlm tags
import bs4
from bs4 import BeautifulSoup

def remove_html(text):
    soup=BeautifulSoup(text,'html.parser')
    return soup.get_text(separator='',strip=True)

sources = ['reuters', 'cnn', 'breitbart', 'fox', 'bbc', 'guardian', 
           'politico', 'npr', 'nytimes', 'washingtonpost', 'ap']

def remove_sources(text):
    for s in sources:
        text = re.sub(r'\b' + re.escape(s) + r'\b', '', text)
    return text


# remove punctuations
import re

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)



# lemitization tokenization urls and numbers
import spacy       

nlp=spacy.load('en_core_web_sm')

def Tokenization(texts):
    results=[]
    for doc in nlp.pipe(texts, batch_size=50, n_process=4):
      Tokens=[]
      for token in doc:
         if token.like_url:
            Tokens.append('url')
         elif token.like_num:
            Tokens.append('num')
         elif token.is_alpha :
            Tokens.append(token.lemma_)
      results.append( " ".join(Tokens))
    return results
        

def preprocess(text):
    if  not isinstance(text,str):
        return ""
     
    text=text.lower()
    text=remove_html(text)
    text=remove_sources(text)
    text=remove_punctuation(text)
    return text

    
df['Combined']=df['Combined'].apply(preprocess)
df['Combined'] = Tokenization(df['Combined'])




x =df['Combined']
y =df['label']
# vectorizing the data

import sklearn
from sklearn.model_selection import train_test_split
x_train,x_test,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf=TfidfVectorizer(max_features=10000,ngram_range=(1,3), sublinear_tf=True,stop_words='english')

xtrain_vec=Tfidf.fit_transform(x_train)
xtest_vec=Tfidf.transform(x_test)

xtrain=xtrain_vec
xtest=xtest_vec
import warnings
warnings.filterwarnings('ignore')

In [2]:
print(100*'=')
print("MultinomialNB Model:\n")
# Model1
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report

Model1=MultinomialNB()
Model1.fit(xtrain,ytrain)
ypred1=Model1.predict(xtest)

# Evaluation
acc=accuracy_score(ytest,ypred1)
print(f"Accuracy:{acc*100:.2f}")

cr=classification_report(ytest,ypred1)
print(f"classification_report:/n{cr}")
# ==============================================================================================
# ==============================================================================================
print(100*'=')
print("LogisticRegression Model:\n")
# Model2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report

Model2=LogisticRegression()
Model2.fit(xtrain,ytrain)
ypred2=Model2.predict(xtest)

# Evaluation
acc=accuracy_score(ytest,ypred2)
print(f"Accuracy:{acc*100:.2f}")
cr=classification_report(ytest,ypred2)
print(f"classification_report:/n{cr}")
# ==============================================================================================
# ==============================================================================================



MultinomialNB Model:

Accuracy:95.01
classification_report:/n              precision    recall  f1-score   support

           0       0.95      0.95      0.95      4702
           1       0.95      0.95      0.95      4236

    accuracy                           0.95      8938
   macro avg       0.95      0.95      0.95      8938
weighted avg       0.95      0.95      0.95      8938

LogisticRegression Model:

Accuracy:98.64
classification_report:/n              precision    recall  f1-score   support

           0       0.99      0.98      0.99      4702
           1       0.98      0.99      0.99      4236

    accuracy                           0.99      8938
   macro avg       0.99      0.99      0.99      8938
weighted avg       0.99      0.99      0.99      8938



In [3]:
print(100*'=')
print("RandomForestClassifier Model:\n")
# Model3
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report

Model3=RandomForestClassifier()
Model3.fit(xtrain,ytrain)
ypred3=Model3.predict(xtest)

# Evaluation
acc=accuracy_score(ytest,ypred3)
print(f"Accuracy:{acc*100:.2f}")
cr=classification_report(ytest,ypred3)
print(f"classification_report:/n{cr}")
# ==============================================================================================
# ==============================================================================================
print(100*'=')
print("LGBMClassifier Model:\n")
# Model5
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score,classification_report

Model4=LGBMClassifier()
Model4.fit(xtrain,ytrain)
ypred4=Model4.predict(xtest)

# Evaluation
acc=accuracy_score(ytest,ypred4)
print(f"Accuracy:{acc*100:.2f}")
cr=classification_report(ytest,ypred4)
print(f"classification_report:/n{cr}")


RandomForestClassifier Model:

Accuracy:98.66
classification_report:/n              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4702
           1       0.99      0.98      0.99      4236

    accuracy                           0.99      8938
   macro avg       0.99      0.99      0.99      8938
weighted avg       0.99      0.99      0.99      8938

LGBMClassifier Model:

[LightGBM] [Info] Number of positive: 16975, number of negative: 18776
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.585015 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1031164
[LightGBM] [Info] Number of data points in the train set: 35751, number of used features: 9995
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.474812 -> initscore=-0.100838
[LightGBM] [Info] Start training from score -0.100838
Accuracy:98.89
classification_report:/n              precision    recall  f1-scor

In [4]:
import joblib
joblib.dump(Model2,'Model2.joblib')
joblib.dump(Tfidf,'Tfidf.joblib')


['Tfidf.joblib']

In [5]:
print("LGBM predictions:", pd.Series(ypred4).value_counts())
print("Logistic predictions:", pd.Series(ypred2).value_counts())
print("NB predictions:", pd.Series(ypred1).value_counts())


LGBM predictions: 0    4707
1    4231
Name: count, dtype: int64
Logistic predictions: 0    4672
1    4266
Name: count, dtype: int64
NB predictions: 0    4700
1    4238
Name: count, dtype: int64


In [8]:
import time
from preprocess import clean_text,fast_tokenize
text = """NASA’s Artemis program has achieved a major milestone..."""
t1 = time.time()
cleantext = clean_text(text)
tokenized = fast_tokenize(cleantext)
t2 = time.time()
print("Preprocessing time:", t2 - t1, "seconds")


Preprocessing time: 0.030211210250854492 seconds
